In [2]:
import mysql.connector
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
conn = mysql.connector.connect(
    host = "*****",
    port = "****",
    user = "****",
    password = "****",
    database = "****"
)

cursor = conn.cursor(buffered=True)

In [7]:
query = """
        create table national_pension_240924 (
            자료생성년월 varchar(10),
            사업장명 varchar(1000),
            사업자등록번호 VARCHAR(100),
            사업장가입상태코드_1_등록_2_탈퇴 VARCHAR(10),
            우편번호 varchar(10),
            사업장지번상세주소 varchar(1000),
            사업장도로명상세주소 varchar(1000),
            고객법정동주소코드 varchar(100),
            고객행정동주소코드 varchar(100),
            법정동주소광역시도코드 varchar(100),
            법정동주소광역시시군구코드 varchar(100),
            법정동주소광역시시군구읍면동코드 varchar(100),
            사업장형태구분코드_1_법인_2_개인 varchar(100),
            사업장업종코드 varchar(100),
            사업장업종코드명 varchar(100),
            적용일자 varchar(10),
            재등록일자 varchar(10),
            탈퇴일자 varchar(10),
            가입자수 numeric(55),
            당월고지금액 numeric(55),
            신규취득자수 numeric(55),
            상실가입자수 numeric(55)
        );
        """

cursor.execute(query)
conn.commit()

In [5]:
load_data_infile = """
        LOAD DATA LOCAL INFILE '/home/cook/ws/myproject/team/national_pension_20240924_utf8.csv'
        INTO TABLE national_pension_240924
        CHARACTER SET utf8mb4
        FIELDS TERMINATED BY ','
        LINES TERMINATED BY '\n'
        IGNORE 1 LINES (
        자료생성년월,
        사업장명,
        사업자등록번호,
        사업장가입상태코드_1_등록_2_탈퇴,
        우편번호,
        사업장지번상세주소,
        사업장도로명상세주소,
        고객법정동주소코드,
        고객행정동주소코드,
        법정동주소광역시도코드,
        법정동주소광역시시군구코드,
        법정동주소광역시시군구읍면동코드,
        사업장형태구분코드_1_법인_2_개인,
        사업장업종코드,
        사업장업종코드명,
        적용일자,
        재등록일자,
        탈퇴일자,
        가입자수,
        당월고지금액,
        신규취득자수,
        상실가입자수);
        """

In [6]:
try:
    cursor.execute(load_data_infile)
    conn.commit()
    print("loaded csv file")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()
finally:
    cursor.close()
    conn.close()

loaded csv file


In [12]:
conn = mysql.connector.connect(
    host = "msdb.cvyy46quatrs.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "choi",
    password = "1234",
    database = "vector",
)

cursor = conn.cursor(buffered=True)

In [4]:
duplicates = 0
successes = 0
noname = 0

cursor.execute("select comp_id, comp_name, employee_num, avg_salary from company where employee_num is null or avg_salary is null")
comps = cursor.fetchall()

for comp in comps:
    comp_id = comp[0]
    comp_name = comp[1]
    employee_num = comp[2]
    avg_salary = comp[3]

    if "(주)" in comp_name:
        comp_name_re = comp_name.replace("(주)", "").strip()
    elif "주식회사" in comp_name:
        comp_name_re = comp_name.replace("주식회사", "").strip()
    else:
        comp_name_re = comp_name

    cursor.execute("""select 사업장명, 가입자수, 당월고지금액 
                   from national_pension_240924 
                   where 사업장명 like %s""", ('%' + comp_name_re + '%',))
    results = cursor.fetchall()

    if len(results) > 1:
        duplicates += 1
        print("----------------------------------------------------------------")
        print(f"Dulicate comp name {comp_name}")
    elif len(results) == 1:
        successes += 1
        print("----------------------------------------------------------------")
        np_employee_num = results[0][1]
        np_month_money = results[0][2]

        if employee_num is None:
            cursor.execute("""update company set employee_num=%s where comp_id=%s""", (np_employee_num, comp_id))
        if avg_salary is None:
            np_avg_salary = (((np_month_money / np_employee_num) / 9) * 100) * 12
            cursor.execute("""update company set avg_salary=%s where comp_id=%s""", (np_avg_salary, comp_id))
    else:
        noname += 1
        print("----------------------------------------------------------------")
        print(f"No such name: {comp_name}")

print(f"duplicates: {duplicates}, successes: {successes}, noname: {noname}")


----------------------------------------------------------------
No such name: (주)비비파트너스
----------------------------------------------------------------
Dulicate comp name (주)긴트
----------------------------------------------------------------
----------------------------------------------------------------
Dulicate comp name (주)테스프
----------------------------------------------------------------
No such name: (주)드림에이치알
----------------------------------------------------------------
----------------------------------------------------------------
----------------------------------------------------------------
----------------------------------------------------------------
Dulicate comp name (주)피엠에프
----------------------------------------------------------------
No such name: (주)휴먼트리월드와이드
----------------------------------------------------------------
----------------------------------------------------------------
No such name: 탑커리어인사이트
--------------------------------------------

In [9]:
conn.commit()
conn.close()

In [25]:
conn = mysql.connector.connect(
    host = "msdb.cvyy46quatrs.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "choi",
    password = "1234",
    database = "vector",
)

cursor = conn.cursor(buffered=True)

dup_noemp = 0
noname = 0

cursor.execute("select comp_id, comp_name, employee_num, avg_salary from company where employee_num is null or avg_salary is null")
comps = cursor.fetchall()

for comp in comps:
    comp_id = comp[0]
    comp_name = comp[1]
    employee_num = comp[2]
    avg_salary = comp[3]
    print("--------------------------------------------")

    if "(주)" in comp_name:
        comp_name_re = comp_name.replace("(주)", "").strip()
    elif "주식회사" in comp_name:
        comp_name_re = comp_name.replace("주식회사", "").strip()
    else:
        comp_name_re = comp_name

    cursor.execute("""select 사업장명, 가입자수, 당월고지금액 
                   from national_pension_240924 
                   where 사업장명 like %s""", ('%' + comp_name_re + '%',))
    results = cursor.fetchall()

    if results:
        if employee_num:
            for r in results:
                np_employee_num = r[1]
                np_month_money = r[2]
                if employee_num == np_employee_num:
                    np_avg_salary = int((((np_month_money / np_employee_num) / 9) * 100) * 12)
                    cursor.execute("""update company set avg_salary=%s where comp_id=%s""", (np_avg_salary, comp_id))
                    print(f"success: {comp_id}, {comp_name}")
                    break
        else:
            print(f"Duplicate, no employee_num comp name: {comp_id}, {comp_name}")
            dup_noemp += 1
    else:
        print(f"No such name: {comp_id}, {comp_name}")
        noname += 1

print(dup_noemp, noname)


--------------------------------------------
No such name: 1, (주)비비파트너스
--------------------------------------------
No such name: 13, (주)드림에이치알
--------------------------------------------
No such name: 19, (주)휴먼트리월드와이드
--------------------------------------------
No such name: 22, 탑커리어인사이트
--------------------------------------------
No such name: 34, 그린에이아이
--------------------------------------------
No such name: 36, (주)투비파트너즈
--------------------------------------------
No such name: 37, (주)피플케어코리아
--------------------------------------------
No such name: 38, 반석써치(주)
--------------------------------------------
No such name: 40, 주식회사 매이드 (MADDE Inc.)
--------------------------------------------
No such name: 47, 주) 석세스코리아컨설팅
--------------------------------------------
Duplicate, no employee_num comp name: 59, SMI
--------------------------------------------
Duplicate, no employee_num comp name: 72, 주식회사 지니어스
--------------------------------------------
No such name: 85, (주)라온서치

In [10]:
dup_noemp, noname

(16, 31)

In [21]:
conn.commit()

In [24]:
conn.close()